# IBM Watson своими руками

https://allennlp.org/models

In [2]:
from allennlp.predictors.predictor import Predictor
import requests
import json
import warnings
import wikipedia
from wikipedia import DisambiguationError
warnings.filterwarnings('ignore')

### Загружаем две модели. 

Первая находит в текстве сущности: актеры, фильмы, компании, географические названия и тд.
Вторая натренирована когда ей дают текст и вопрос по этому тексту - находить где в текстве будет ответ.

In [ ]:
!wget "https://s3-us-west-2.amazonaws.com/allennlp/models/ner-model-2018.04.26.tar.gz"
!wget "https://s3-us-west-2.amazonaws.com/allennlp/models/bidaf-model-2017.09.15-charpad.tar.gz"

In [4]:
ner = predictor = Predictor.from_path("ner-model-2018.04.26.tar.gz")
bidaf = Predictor.from_path("bidaf-model-2017.09.15-charpad.tar.gz")

Эта функция достанет из вопроса "сущности"

In [6]:
def extract_terms(question):
    entities = ner.predict(
      sentence=question
    )
    terms = []
    current_term = ''
    for tag, word in zip(entities['tags'], entities['words']):
        if tag[0] == 'U':
            terms.append(word)
        elif tag[0] == 'B':
            current_term = word
        elif tag[0] == 'I':
            current_term += word
        elif tag[0] == 'L':
            terms.append(current_term + ' ' + word)
    return terms

Эта функция скачает в википедии страницы для найденных сущностей

In [7]:
def search_wiki(terms):
    full_text = []
    for term in terms:
        try:
            full_text.append(wikipedia.page(term).content)
        except DisambiguationError:
            pages = wikipedia.search(term, results=2)
            if term in pages: pages.remove(term)
            for page in pages:
                full_text.append(wikipedia.page(page).content)
    return '\n'.join(full_text)

# Задаем вопрос

In [9]:
print("Please enter you question:")
question = input()

search_terms = extract_terms(question)
print('Searching for terms:')
print(search_terms)
print()
wiki_text = search_wiki(search_terms)
print('Short exract from wiki text:')
print(wiki_text[:50] + '...')
print()

print('Computing the answer...')
print()

answer = bidaf.predict(
  passage=wiki_text,
  question=question
)
print('The answer is (probably):')
print(answer['best_span_str'])

Please enter you question:
What is the fourth planet from the Sun?
Searching for terms:
['Sun']

Short exract from wiki text:
The Sun is the star at the center of the Solar Sys...

Computing the answer...

The answer is (probably):
Mars
